# Human Activity Recognition - SSL JOINT Consistency

Last Update : 31 July 2019

In [1]:
N_THREADS = 8
# Nota Bene : notebooks don't deallocate GPU memory
IS_FORCE_CPU = False # can also be set in the trainer

## Environment

In [2]:
cd ..

/conv


In [3]:
%autosave 600
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# CENTER PLOTS
from IPython.core.display import HTML
display(HTML(""" <style> .output_png {display: table-cell; text-align: center; margin:auto; }
.prompt display:none;}  </style>"""))

import os
if IS_FORCE_CPU:
    os.environ['CUDA_VISIBLE_DEVICES'] = ""

import sys
sys.path.append("notebooks")

import numpy as np
import matplotlib.pyplot as plt
from functools import partial
import pandas as pd
import h5py


import torch
torch.set_num_threads(N_THREADS)

Autosaving every 600 seconds


# Dataset

In [4]:
from skssl.transformers.neuralproc.datasplit import CntxtTrgtGetter, GetRandomIndcs, get_all_indcs
from utils.data.tsdata import get_timeseries_dataset, SparseMultiTimeSeriesDataset

get_cntxt_trgt_test = CntxtTrgtGetter(contexts_getter=GetRandomIndcs(min_n_indcs=0.1, max_n_indcs=0.5),
                                     targets_getter=get_all_indcs,
                                     is_add_cntxts_to_trgts=False)  # don't context points to tagrtes

get_cntxt_trgt_feat = CntxtTrgtGetter(contexts_getter=get_all_indcs,
                                     targets_getter=get_all_indcs,
                                     is_add_cntxts_to_trgts=False)  # don't context points to tagrtes

get_cntxt_trgt = CntxtTrgtGetter(contexts_getter=GetRandomIndcs(min_n_indcs=0.01, max_n_indcs=0.99),
                                 targets_getter=GetRandomIndcs(min_n_indcs=0.5, max_n_indcs=0.99),
                                 is_add_cntxts_to_trgts=False)  # don't context points to tagrtes

In [5]:
data_both = get_timeseries_dataset("har")(split="both")

def cntxt_trgt_collate(get_cntxt_trgt, is_repeat_batch=False):
    def mycollate(batch):
        min_length = min([v.size(0) for b in batch for k,v in b[0].items() if "X" in k])
        # chose first min_legth of each (assumes that randomized)
        
        batch = [({k:v[:min_length, ...] for k,v in b[0].items()}, b[1]) for b in batch]        
        collated = torch.utils.data.dataloader.default_collate(batch)
        
        X = collated[0]["X"]
        y = collated[0]["y"]
        
        if is_repeat_batch:
            
            X = torch.cat([X,X], dim=0)
            y = torch.cat([y,y], dim=0)
            collated[1] = torch.cat([collated[1], collated[1]], dim=0) # targets
        
        collated[0]["X"], collated[0]["y"], collated[0]["X_trgt"], collated[0]["y_trgt"] = get_cntxt_trgt(X, y)
        
        return collated
    return mycollate

In [6]:
X_DIM = 1  # 1D spatial input (although actually 2 but the first is for sparse channels)
Y_DIM = data_both.data.shape[-1] # multiple channels
N_TARGETS = len(np.unique(data_both.targets))

sampling_percentages = [0.05, 0.1, 0.3, 0.5, 0.7, 1]
label_percentages = [N_TARGETS, N_TARGETS*2, 0.01, 0.05, 0.1, 0.3, 0.5, 1]

## Model

In [7]:
import torch.nn as nn
from skssl.transformers import GlobalNeuralProcess, NeuralProcessLoss, AttentiveNeuralProcess, NeuralProcessSSLLoss
from skssl.utils.helpers import rescale_range
from skssl.predefined import UnetCNN, CNN, MLP, SparseSetConv, SetConv, MlpRBF, GaussianRBF, BatchSparseSetConv
from skssl.transformers.neuralproc.datasplit import precomputed_cntxt_trgt_split
from utils.helpers import count_parameters

In [8]:
from copy import deepcopy

models = {}

min_std=5e-3

unet = partial(UnetCNN,
               Conv=torch.nn.Conv1d,
               Pool=torch.nn.MaxPool1d,
               upsample_mode="linear",
               n_layers=18,
               is_double_conv=True,
               is_depth_separable=True,
               Normalization=torch.nn.BatchNorm1d,
               is_chan_last=True,
               bottleneck=None,
               kernel_size=7,
               max_nchannels=256,
              is_force_same_bottleneck=True,
               _is_summary=True,
              )

kwargs = dict(x_dim=X_DIM, 
              y_dim=Y_DIM,
              min_std=min_std,
                n_tmp_queries=128,
                r_dim=64,
              keys_to_tmp_attn=partial(SetConv, RadialBasisFunc=GaussianRBF),
              TmpSelfAttn=unet,
              tmp_to_queries_attn=partial(SetConv, RadialBasisFunc=GaussianRBF),
              is_skip_tmp=False,
              is_use_x=False,
              get_cntxt_trgt=precomputed_cntxt_trgt_split,
              is_encode_xy=False,
             Classifier=partial(MLP, input_size=256+Y_DIM*4, output_size=N_TARGETS, 
                                dropout=0., hidden_size=128, n_hidden_layers=3, is_res=True))

models["ssl_classifier_gnp_large_shared_bottleneck"] = partial(GlobalNeuralProcess, **kwargs)

kwargs_bis = deepcopy(kwargs)
kwargs_bis["Classifier"] = None

models["transformer_gnp_large_shared_bottleneck"] = partial(GlobalNeuralProcess, **kwargs_bis)

In [9]:
from utils.helpers import count_parameters
for k,v in models.items():
    print(k, "- N Param:", count_parameters(v()))

ssl_classifier_gnp_large_shared_bottleneck - N Param: 1078238
transformer_gnp_large_shared_bottleneck - N Param: 1006936


In [10]:
def load_pretrained_(models, sampling_percentages):
    # ALREADY INITALIZE TO BE ABLE TO LOAD
    models["ssl_classifier_gnp_large_shared_bottleneck"] = partial(GlobalNeuralProcess, **kwargs)()

    kwargs_bis = deepcopy(kwargs)
    kwargs_bis["Classifier"] = None

    models["transformer_gnp_large_shared_bottleneck"] = partial(GlobalNeuralProcess, **kwargs_bis)()

    # load all transformers
    loaded_models = {}
    for sampling_perc in sampling_percentages:
        for k, m in models.items():
            if "transformer" not in k:
                continue

            out = train_models_({"{}%har".format(int(sampling_perc*100)): 
                                                (None, None)}, 
                                  {k :m },
                                   chckpnt_dirname=chckpnt_dirname_old,
                                seed=None,
                                   is_retrain=False)

            pretrained_model = out[list(out.keys())[0]].module_
            model_dict = models[k.replace("transformer", "ssl_classifier")].state_dict()
            model_dict.update(pretrained_model.state_dict())
            models[k.replace("transformer", "ssl_classifier")].load_state_dict(model_dict)

# Hyperparameter Optimisation


In [11]:
from ntbks_helpers import train_models_
from skorch.dataset import CVSplit
from utils.data.ssldata import get_train_dev_test_ssl
import random

N_EPOCHS = 100 
BATCH_SIZE = 32
IS_RETRAIN = False # if false load precomputed
chckpnt_dirname_old="results/challenge/har/"
chckpnt_dirname="results/challenge/har_new/"

from skssl.utils.helpers import HyperparameterInterpolator

n_steps_per_epoch = len(data_both)//BATCH_SIZE
get_lambda_clf=HyperparameterInterpolator(1, 10, N_EPOCHS*n_steps_per_epoch, mode="linear")

# No Augment

In [12]:
from skorch.callbacks import Freezer, LRScheduler

data_trainers = {}

for run in range(3):
    for sampling_perc in [1]:
        for label_perc in [0.01]:
            load_pretrained_(models, [sampling_perc])

            get_lambda_clf=HyperparameterInterpolator(1, 50, N_EPOCHS*n_steps_per_epoch, mode="linear")

            data_train, _, data_test = get_train_dev_test_ssl("har", 
                                                              n_labels=label_perc, 
                                                              data_perc=sampling_perc, 
                                                              dev_size=0,
                                                              seed=random.randint(0,10000),
                                                              is_augment=False)
            print(len(data_train))

            # add test as unlabeled data
            data_train.data = np.concatenate([data_train.data, data_test.data], axis=0)
            data_train.targets = np.concatenate([data_train.targets, -1*np.ones_like(data_test.targets)], axis=0)
            data_train.indcs = np.concatenate([data_train.indcs, data_test.indcs], axis=0)

            data_trainers.update(train_models_({"har{}%_lab{}%_run{}".format(int(sampling_perc*100), int(label_perc*100), run): 
                                                (data_train, data_test)}, 
                                  {k + "_finetune_noaug" :m for k,m in models.items() if "ssl_classifier" in k}, 
                                  criterion=partial(NeuralProcessSSLLoss, 
                                                    get_lambda_sup=lambda: get_lambda_clf(True),
                                                    n_max_elements=int(128*sampling_perc),
                                                    label_perc=(label_perc * data_train.n_train)/data_train.n_total, # label perc is lower ebcause cocnat to test
                                                    min_sigma=min_std,
                                                    is_unsup_forall=False,
                                                    is_ssl_only=False,
                                                    get_lambda_unsup=lambda: 1,
                                                     get_lambda_ent=lambda: 0.5,  # both do something similar
                                                     get_lambda_neg_cons=lambda: 0.5,
                                                    ),
                                    patience=15,
                                  chckpnt_dirname=chckpnt_dirname,
                                  max_epochs=N_EPOCHS,
                                  batch_size=BATCH_SIZE,
                                  is_retrain=IS_RETRAIN,
                                    is_monitor_acc=True,
                                  callbacks=[],
                                  iterator_train__collate_fn=cntxt_trgt_collate(get_cntxt_trgt, is_repeat_batch=True),  
                                  iterator_valid__collate_fn=cntxt_trgt_collate(get_cntxt_trgt_feat),
                                              ))



--- Loading 100%har/transformer_gnp_large_shared_bottleneck ---

100%har/transformer_gnp_large_shared_bottleneck best epoch: 86 val_loss: -8.16725208180622
7352

--- Loading har100%_lab1%_run0/ssl_classifier_gnp_large_shared_bottleneck_finetune_noaug ---

har100%_lab1%_run0/ssl_classifier_gnp_large_shared_bottleneck_finetune_noaug best epoch: 1 val_loss: 0.5792485174988745

--- Loading 100%har/transformer_gnp_large_shared_bottleneck ---

100%har/transformer_gnp_large_shared_bottleneck best epoch: 86 val_loss: -8.16725208180622
7352

--- Loading har100%_lab1%_run1/ssl_classifier_gnp_large_shared_bottleneck_finetune_noaug ---

har100%_lab1%_run1/ssl_classifier_gnp_large_shared_bottleneck_finetune_noaug best epoch: 4 val_loss: 0.534082076773629

--- Loading 100%har/transformer_gnp_large_shared_bottleneck ---

100%har/transformer_gnp_large_shared_bottleneck best epoch: 86 val_loss: -8.16725208180622
7352

--- Loading har100%_lab1%_run2/ssl_classifier_gnp_large_shared_bottleneck_finetune_n

In [13]:
import pandas as pd

out = pd.Series({k:v.history[-1]["valid_acc"] for k,v in data_trainers.items()}).reset_index(name="accuracy")
splitted = out["index"].str.split("/", expand = True)
out["meta"] = splitted[0]
out["models"] = splitted[1]

splitted2 = out["meta"].str.split("_run", expand = True)
out["meta"] = splitted2[0]
out["run"] = splitted2[1]

splitted3 = out["meta"].str.split("_lab", expand = True)
out["data sample"] = splitted3[0].str.split("har", expand = True)[1]
out["lab"] = splitted3[1]


out.drop(columns =["index"], inplace = True) 

out.groupby(["models", "lab", "data sample"]).describe()

accuracy  \
                                                                      count   
models                                             lab data sample            
ssl_classifier_gnp_large_shared_bottleneck_fine... 1%  100%             3.0   

                                                                              \
                                                                        mean   
models                                             lab data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 1%  100%         0.870264   

                                                                              \
                                                                         std   
models                                             lab data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 1%  100%         0.003135   

                                                                              \
                                                                         min   
models                                             lab data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 1%  100%         0.866644   

                                                                              \
                                                                         25%   
models                                             lab data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 1%  100%         0.869359   

                                                                              \
                                                                         50%   
models                                             lab data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 1%  100%         0.872073   

                                                                              \
                                                                         75%   
models                                             lab data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 1%  100%         0.872073   

                                                                              
                                                                         max  
models                                             lab data sample            
ssl_classifier_gnp_large_shared_bottleneck_fine... 1%  100%         0.872073

# All

In [14]:
from skorch.callbacks import Freezer, LRScheduler


data_trainers = {}

for run in range(3):
    for sampling_perc in [1]:
        for label_perc in [0.01]:
            load_pretrained_(models, [sampling_perc])

            get_lambda_clf=HyperparameterInterpolator(1, 50, N_EPOCHS*n_steps_per_epoch, mode="linear")

            data_train, _, data_test = get_train_dev_test_ssl("har", 
                                                              n_labels=label_perc, 
                                                              data_perc=sampling_perc, 
                                                              dev_size=0,
                                                              seed=random.randint(0,10000),
                                                              is_augment=True)
            print(len(data_train))

            # add test as unlabeled data
            data_train.data = np.concatenate([data_train.data, data_test.data], axis=0)
            data_train.targets = np.concatenate([data_train.targets, -1*np.ones_like(data_test.targets)], axis=0)
            data_train.indcs = np.concatenate([data_train.indcs, data_test.indcs], axis=0)

            data_trainers.update(train_models_({"har{}%_lab{}%_run{}".format(int(sampling_perc*100), int(label_perc*100), run): 
                                                (data_train, data_test)}, 
                                  {k + "_finetune" :m for k,m in models.items() if "ssl_classifier" in k}, 
                                  criterion=partial(NeuralProcessSSLLoss, 
                                                    get_lambda_sup=lambda: get_lambda_clf(True),
                                                    n_max_elements=int(128*sampling_perc),
                                                    label_perc=(label_perc * data_train.n_train)/data_train.n_total, # label perc is lower ebcause cocnat to test
                                                    min_sigma=min_std,
                                                    is_unsup_forall=False,
                                                    is_ssl_only=False,
                                                    get_lambda_unsup=lambda: 1,
                                                     get_lambda_ent=lambda: 0.5,  # both do something similar
                                                     get_lambda_neg_cons=lambda: 0.5,
                                                    ),
                                    patience=15,
                                  chckpnt_dirname=chckpnt_dirname,
                                  max_epochs=N_EPOCHS,
                                  batch_size=BATCH_SIZE,
                                  is_retrain=IS_RETRAIN,
                                    is_monitor_acc=True,
                                  callbacks=[],
                                  iterator_train__collate_fn=cntxt_trgt_collate(get_cntxt_trgt, is_repeat_batch=True),  
                                  iterator_valid__collate_fn=cntxt_trgt_collate(get_cntxt_trgt_feat),
                                              ))



--- Loading 100%har/transformer_gnp_large_shared_bottleneck ---

100%har/transformer_gnp_large_shared_bottleneck best epoch: 86 val_loss: -8.16725208180622
14530

--- Loading har100%_lab1%_run0/ssl_classifier_gnp_large_shared_bottleneck_finetune ---

har100%_lab1%_run0/ssl_classifier_gnp_large_shared_bottleneck_finetune best epoch: 4 val_loss: 0.7066817718074813

--- Loading 100%har/transformer_gnp_large_shared_bottleneck ---

100%har/transformer_gnp_large_shared_bottleneck best epoch: 86 val_loss: -8.16725208180622
14530

--- Loading har100%_lab1%_run1/ssl_classifier_gnp_large_shared_bottleneck_finetune ---

har100%_lab1%_run1/ssl_classifier_gnp_large_shared_bottleneck_finetune best epoch: 1 val_loss: 0.7923410251903502

--- Loading 100%har/transformer_gnp_large_shared_bottleneck ---

100%har/transformer_gnp_large_shared_bottleneck best epoch: 86 val_loss: -8.16725208180622
14530

--- Loading har100%_lab1%_run2/ssl_classifier_gnp_large_shared_bottleneck_finetune ---

har100%_lab1%_ru

In [15]:
import pandas as pd

out = pd.Series({k:v.history[-1]["valid_acc"] for k,v in data_trainers.items()}).reset_index(name="accuracy")
splitted = out["index"].str.split("/", expand = True)
out["meta"] = splitted[0]
out["models"] = splitted[1]

splitted2 = out["meta"].str.split("_run", expand = True)
out["meta"] = splitted2[0]
out["run"] = splitted2[1]

splitted3 = out["meta"].str.split("_lab", expand = True)
out["data sample"] = splitted3[0].str.split("har", expand = True)[1]
out["lab"] = splitted3[1]


out.drop(columns =["index"], inplace = True) 

out.groupby(["models", "lab", "data sample"]).describe()

accuracy  \
                                                                      count   
models                                             lab data sample            
ssl_classifier_gnp_large_shared_bottleneck_fine... 1%  100%             3.0   

                                                                              \
                                                                        mean   
models                                             lab data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 1%  100%         0.883497   

                                                                              \
                                                                         std   
models                                             lab data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 1%  100%         0.008286   

                                                                              \
                                                                         min   
models                                             lab data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 1%  100%         0.876824   

                                                                             \
                                                                        25%   
models                                             lab data sample            
ssl_classifier_gnp_large_shared_bottleneck_fine... 1%  100%         0.87886   

                                                                              \
                                                                         50%   
models                                             lab data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 1%  100%         0.880896   

                                                                              \
                                                                         75%   
models                                             lab data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 1%  100%         0.886834   

                                                                              
                                                                         max  
models                                             lab data sample            
ssl_classifier_gnp_large_shared_bottleneck_fine... 1%  100%         0.892772

## Without Neg Consistency

In [16]:
from skorch.callbacks import Freezer, LRScheduler

data_trainers = {}

for run in range(3):
    for sampling_perc in [1]:
        for label_perc in [0.01]:
            load_pretrained_(models, [sampling_perc])

            get_lambda_clf=HyperparameterInterpolator(1, 50, N_EPOCHS*n_steps_per_epoch, mode="linear")

            data_train, _, data_test = get_train_dev_test_ssl("har", 
                                                              n_labels=label_perc, 
                                                              data_perc=sampling_perc, 
                                                              dev_size=0,
                                                              seed=random.randint(0,10000),
                                                              is_augment=True)

            # add test as unlabeled data
            data_train.data = np.concatenate([data_train.data, data_test.data], axis=0)
            data_train.targets = np.concatenate([data_train.targets, -1*np.ones_like(data_test.targets)], axis=0)
            data_train.indcs = np.concatenate([data_train.indcs, data_test.indcs], axis=0)

            data_trainers.update(train_models_({"har{}%_lab{}%_run{}".format(int(sampling_perc*100), int(label_perc*100), run):  
                                                (data_train, data_test)}, 
                                  {k + "_finetune_nonegcons" :m for k,m in models.items() if "ssl_classifier" in k}, 
                                  criterion=partial(NeuralProcessSSLLoss, 
                                                    get_lambda_sup=lambda: get_lambda_clf(True),
                                                    n_max_elements=int(128*sampling_perc),
                                                    label_perc=(label_perc * data_train.n_train)/data_train.n_total, # label perc is lower ebcause cocnat to test
                                                    min_sigma=min_std,
                                                    is_unsup_forall=False,
                                                    is_ssl_only=False,
                                                    get_lambda_unsup=lambda: 1,
                                                     get_lambda_ent=lambda: 1,  # both do something similar
                                                     get_lambda_neg_cons=lambda: 0.,
                                                    ),
                                    patience=15,
                                  chckpnt_dirname=chckpnt_dirname,
                                  max_epochs=N_EPOCHS,
                                  batch_size=BATCH_SIZE,
                                  is_retrain=IS_RETRAIN,
                                    is_monitor_acc=True,
                                  callbacks=[],
                                  iterator_train__collate_fn=cntxt_trgt_collate(get_cntxt_trgt, is_repeat_batch=True),  
                                  iterator_valid__collate_fn=cntxt_trgt_collate(get_cntxt_trgt_feat),
                                              ))



--- Loading 100%har/transformer_gnp_large_shared_bottleneck ---

100%har/transformer_gnp_large_shared_bottleneck best epoch: 86 val_loss: -8.16725208180622

--- Loading har100%_lab1%_run0/ssl_classifier_gnp_large_shared_bottleneck_finetune_nonegcons ---

har100%_lab1%_run0/ssl_classifier_gnp_large_shared_bottleneck_finetune_nonegcons best epoch: 1 val_loss: 0.5842806758175149

--- Loading 100%har/transformer_gnp_large_shared_bottleneck ---

100%har/transformer_gnp_large_shared_bottleneck best epoch: 86 val_loss: -8.16725208180622

--- Loading har100%_lab1%_run1/ssl_classifier_gnp_large_shared_bottleneck_finetune_nonegcons ---

har100%_lab1%_run1/ssl_classifier_gnp_large_shared_bottleneck_finetune_nonegcons best epoch: 1 val_loss: 0.6183924950055038

--- Loading 100%har/transformer_gnp_large_shared_bottleneck ---

100%har/transformer_gnp_large_shared_bottleneck best epoch: 86 val_loss: -8.16725208180622

--- Loading har100%_lab1%_run2/ssl_classifier_gnp_large_shared_bottleneck_finetune

In [17]:
import pandas as pd

out = pd.Series({k:v.history[-1]["valid_acc"] for k,v in data_trainers.items()}).reset_index(name="accuracy")
splitted = out["index"].str.split("/", expand = True)
out["meta"] = splitted[0]
out["models"] = splitted[1]

splitted2 = out["meta"].str.split("_run", expand = True)
out["meta"] = splitted2[0]
out["run"] = splitted2[1]

splitted3 = out["meta"].str.split("_lab", expand = True)
out["data sample"] = splitted3[0].str.split("har", expand = True)[1]
out["lab"] = splitted3[1]


out.drop(columns =["index"], inplace = True) 

out.groupby(["models", "lab", "data sample"]).describe()

accuracy  \
                                                                      count   
models                                             lab data sample            
ssl_classifier_gnp_large_shared_bottleneck_fine... 1%  100%             3.0   

                                                                              \
                                                                        mean   
models                                             lab data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 1%  100%         0.884968   

                                                                             \
                                                                        std   
models                                             lab data sample            
ssl_classifier_gnp_large_shared_bottleneck_fine... 1%  100%         0.00445   

                                                                              \
                                                                         min   
models                                             lab data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 1%  100%         0.880896   

                                                                              \
                                                                         25%   
models                                             lab data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 1%  100%         0.882592   

                                                                              \
                                                                         50%   
models                                             lab data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 1%  100%         0.884289   

                                                                              \
                                                                         75%   
models                                             lab data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 1%  100%         0.887004   

                                                                              
                                                                         max  
models                                             lab data sample            
ssl_classifier_gnp_large_shared_bottleneck_fine... 1%  100%         0.889718

Show number of steps for convergence becauseprobably no unsup is fine but not improving

# No Entropy

In [18]:
from skorch.callbacks import Freezer, LRScheduler

data_trainers = {}

for run in range(3):
    for sampling_perc in [1]:
        for label_perc in [0.01]:
            load_pretrained_(models, [sampling_perc])

            get_lambda_clf=HyperparameterInterpolator(1, 50, N_EPOCHS*n_steps_per_epoch, mode="linear")

            data_train, _, data_test = get_train_dev_test_ssl("har", 
                                                              n_labels=label_perc, 
                                                              data_perc=sampling_perc, 
                                                              dev_size=0,
                                                              seed=random.randint(0,10000),
                                                              is_augment=True)

            # add test as unlabeled data
            data_train.data = np.concatenate([data_train.data, data_test.data], axis=0)
            data_train.targets = np.concatenate([data_train.targets, -1*np.ones_like(data_test.targets)], axis=0)
            data_train.indcs = np.concatenate([data_train.indcs, data_test.indcs], axis=0)

            data_trainers.update(train_models_({"har{}%_lab{}%_run{}".format(int(sampling_perc*100), int(label_perc*100), run):
                                                (data_train, data_test)}, 
                                  {k + "_finetune_noent" :m for k,m in models.items() if "ssl_classifier" in k}, 
                                  criterion=partial(NeuralProcessSSLLoss, 
                                                    get_lambda_sup=lambda: get_lambda_clf(True),
                                                    n_max_elements=int(128*sampling_perc),
                                                    label_perc=(label_perc * data_train.n_train)/data_train.n_total, # label perc is lower ebcause cocnat to test
                                                    min_sigma=min_std,
                                                    is_unsup_forall=False,
                                                    is_ssl_only=False,
                                                    get_lambda_unsup=lambda: 1,
                                                     get_lambda_ent=lambda: 0,  # both do something similar
                                                     get_lambda_neg_cons=lambda: 1.,
                                                    ),
                                    patience=15,
                                  chckpnt_dirname=chckpnt_dirname,
                                  max_epochs=N_EPOCHS,
                                  batch_size=BATCH_SIZE,
                                  is_retrain=IS_RETRAIN,
                                    is_monitor_acc=True,
                                  callbacks=[],
                                  iterator_train__collate_fn=cntxt_trgt_collate(get_cntxt_trgt, is_repeat_batch=True),  
                                  iterator_valid__collate_fn=cntxt_trgt_collate(get_cntxt_trgt_feat),
                                              ))



--- Loading 100%har/transformer_gnp_large_shared_bottleneck ---

100%har/transformer_gnp_large_shared_bottleneck best epoch: 86 val_loss: -8.16725208180622

--- Loading har100%_lab1%_run0/ssl_classifier_gnp_large_shared_bottleneck_finetune_noent ---

har100%_lab1%_run0/ssl_classifier_gnp_large_shared_bottleneck_finetune_noent best epoch: 1 val_loss: 0.6706024456477223

--- Loading 100%har/transformer_gnp_large_shared_bottleneck ---

100%har/transformer_gnp_large_shared_bottleneck best epoch: 86 val_loss: -8.16725208180622

--- Loading har100%_lab1%_run1/ssl_classifier_gnp_large_shared_bottleneck_finetune_noent ---

har100%_lab1%_run1/ssl_classifier_gnp_large_shared_bottleneck_finetune_noent best epoch: 1 val_loss: 0.6930868627258914

--- Loading 100%har/transformer_gnp_large_shared_bottleneck ---

100%har/transformer_gnp_large_shared_bottleneck best epoch: 86 val_loss: -8.16725208180622

--- Loading har100%_lab1%_run2/ssl_classifier_gnp_large_shared_bottleneck_finetune_noent ---

har1

In [19]:
import pandas as pd

out = pd.Series({k:v.history[-1]["valid_acc"] for k,v in data_trainers.items()}).reset_index(name="accuracy")
splitted = out["index"].str.split("/", expand = True)
out["meta"] = splitted[0]
out["models"] = splitted[1]

splitted2 = out["meta"].str.split("_run", expand = True)
out["meta"] = splitted2[0]
out["run"] = splitted2[1]

splitted3 = out["meta"].str.split("_lab", expand = True)
out["data sample"] = splitted3[0].str.split("har", expand = True)[1]
out["lab"] = splitted3[1]


out.drop(columns =["index"], inplace = True) 

out.groupby(["models", "lab", "data sample"]).describe()

accuracy  \
                                                                      count   
models                                             lab data sample            
ssl_classifier_gnp_large_shared_bottleneck_fine... 1%  100%             3.0   

                                                                              \
                                                                        mean   
models                                             lab data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 1%  100%         0.881235   

                                                                              \
                                                                         std   
models                                             lab data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 1%  100%         0.002562   

                                                                              \
                                                                         min   
models                                             lab data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 1%  100%         0.878521   

                                                                              \
                                                                         25%   
models                                             lab data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 1%  100%         0.880048   

                                                                              \
                                                                         50%   
models                                             lab data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 1%  100%         0.881574   

                                                                              \
                                                                         75%   
models                                             lab data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 1%  100%         0.882592   

                                                                             
                                                                        max  
models                                             lab data sample           
ssl_classifier_gnp_large_shared_bottleneck_fine... 1%  100%         0.88361

## No Unsup

In [22]:
from skorch.callbacks import Freezer, LRScheduler

data_trainers = {}

for run in range(3):
    for sampling_perc in [1]:
        for label_perc in [0.01]:
            load_pretrained_(models, [sampling_perc])

            get_lambda_clf=HyperparameterInterpolator(1, 50, N_EPOCHS*n_steps_per_epoch, mode="linear")

            data_train, _, data_test = get_train_dev_test_ssl("har", 
                                                              n_labels=label_perc, 
                                                              data_perc=sampling_perc, 
                                                              seed=random.randint(0,10000),
                                                              dev_size=0,
                                                              is_augment=True)

            # add test as unlabeled data
            data_train.data = np.concatenate([data_train.data, data_test.data], axis=0)
            data_train.targets = np.concatenate([data_train.targets, -1*np.ones_like(data_test.targets)], axis=0)
            data_train.indcs = np.concatenate([data_train.indcs, data_test.indcs], axis=0)

            data_trainers.update(train_models_({"har{}%_lab{}%_run{}".format(int(sampling_perc*100), int(label_perc*100), run):
                                                (data_train, data_test)}, 
                                  {k + "_finetune_nounsup" :m for k,m in models.items() if "ssl_classifier" in k}, 
                                  criterion=partial(NeuralProcessSSLLoss, 
                                                    get_lambda_sup=lambda: get_lambda_clf(True),
                                                    n_max_elements=int(128*sampling_perc),
                                                    label_perc=(label_perc * data_train.n_train)/data_train.n_total, # label perc is lower ebcause cocnat to test
                                                    min_sigma=min_std,
                                                    is_unsup_forall=False,
                                                    is_ssl_only=False,
                                                    get_lambda_unsup=lambda: 0,
                                                     get_lambda_ent=lambda: 0.5,  # both do something similar
                                                     get_lambda_neg_cons=lambda: .5,
                                                    ),
                                    patience=15,
                                  chckpnt_dirname=chckpnt_dirname,
                                  max_epochs=N_EPOCHS,
                                  batch_size=BATCH_SIZE,
                                  is_retrain=IS_RETRAIN,
                                    is_monitor_acc=True,
                                  callbacks=[],
                                  iterator_train__collate_fn=cntxt_trgt_collate(get_cntxt_trgt, is_repeat_batch=True),  
                                  iterator_valid__collate_fn=cntxt_trgt_collate(get_cntxt_trgt_feat),
                                              ))



--- Loading 100%har/transformer_gnp_large_shared_bottleneck ---

100%har/transformer_gnp_large_shared_bottleneck best epoch: 86 val_loss: -8.16725208180622

--- Loading har100%_lab1%_run0/ssl_classifier_gnp_large_shared_bottleneck_finetune_nounsup ---

har100%_lab1%_run0/ssl_classifier_gnp_large_shared_bottleneck_finetune_nounsup best epoch: 1 val_loss: 0.6362027577476903

--- Loading 100%har/transformer_gnp_large_shared_bottleneck ---

100%har/transformer_gnp_large_shared_bottleneck best epoch: 86 val_loss: -8.16725208180622

--- Loading har100%_lab1%_run1/ssl_classifier_gnp_large_shared_bottleneck_finetune_nounsup ---

har100%_lab1%_run1/ssl_classifier_gnp_large_shared_bottleneck_finetune_nounsup best epoch: 1 val_loss: 0.6330289104488853

--- Loading 100%har/transformer_gnp_large_shared_bottleneck ---

100%har/transformer_gnp_large_shared_bottleneck best epoch: 86 val_loss: -8.16725208180622

--- Loading har100%_lab1%_run2/ssl_classifier_gnp_large_shared_bottleneck_finetune_nounsup

In [23]:
import pandas as pd

out = pd.Series({k:v.history[-1]["valid_acc"] for k,v in data_trainers.items()}).reset_index(name="accuracy")
splitted = out["index"].str.split("/", expand = True)
out["meta"] = splitted[0]
out["models"] = splitted[1]

splitted2 = out["meta"].str.split("_run", expand = True)
out["meta"] = splitted2[0]
out["run"] = splitted2[1]

splitted3 = out["meta"].str.split("_lab", expand = True)
out["data sample"] = splitted3[0].str.split("har", expand = True)[1]
out["lab"] = splitted3[1]


out.drop(columns =["index"], inplace = True) 

out.groupby(["models", "lab", "data sample"]).describe()

accuracy  \
                                                                      count   
models                                             lab data sample            
ssl_classifier_gnp_large_shared_bottleneck_fine... 1%  100%             3.0   

                                                                              \
                                                                        mean   
models                                             lab data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 1%  100%         0.871621   

                                                                              \
                                                                         std   
models                                             lab data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 1%  100%         0.004632   

                                                                              \
                                                                         min   
models                                             lab data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 1%  100%         0.866305   

                                                                              \
                                                                         25%   
models                                             lab data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 1%  100%         0.870037   

                                                                             \
                                                                        50%   
models                                             lab data sample            
ssl_classifier_gnp_large_shared_bottleneck_fine... 1%  100%         0.87377   

                                                                              \
                                                                         75%   
models                                             lab data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 1%  100%         0.874279   

                                                                              
                                                                         max  
models                                             lab data sample            
ssl_classifier_gnp_large_shared_bottleneck_fine... 1%  100%         0.874788

#  SSL Only

In [24]:
from skorch.callbacks import Freezer, LRScheduler

data_trainers = {}

for run in range(3):
    for sampling_perc in [1]:
        for label_perc in [0.01]:
            load_pretrained_(models, [sampling_perc])

            get_lambda_clf=HyperparameterInterpolator(1, 50, N_EPOCHS*n_steps_per_epoch, mode="linear")

            data_train, _, data_test = get_train_dev_test_ssl("har", 
                                                              n_labels=label_perc, 
                                                              data_perc=sampling_perc,
                                                              seed=random.randint(0,10000),
                                                              dev_size=0,
                                                              is_augment=True)

            # add test as unlabeled data
            data_train.data = np.concatenate([data_train.data, data_test.data], axis=0)
            data_train.targets = np.concatenate([data_train.targets, -1*np.ones_like(data_test.targets)], axis=0)
            data_train.indcs = np.concatenate([data_train.indcs, data_test.indcs], axis=0)

            data_trainers.update(train_models_({"har{}%_lab{}%_run{}".format(int(sampling_perc*100), int(label_perc*100), run):
                                                (data_train, data_test)}, 
                                  {k + "_finetune_sslonly" :m for k,m in models.items() if "ssl_classifier" in k}, 
                                  criterion=partial(NeuralProcessSSLLoss, 
                                                    get_lambda_sup=lambda: get_lambda_clf(True),
                                                    n_max_elements=int(128*sampling_perc),
                                                    label_perc=(label_perc * data_train.n_train)/data_train.n_total, # label perc is lower ebcause cocnat to test
                                                    min_sigma=min_std,
                                                    is_unsup_forall=False,
                                                    is_ssl_only=True,
                                                    get_lambda_unsup=lambda: 1,
                                                     get_lambda_ent=lambda: 0.5,  # both do something similar
                                                     get_lambda_neg_cons=lambda: .5,
                                                    ),
                                    patience=15,
                                  chckpnt_dirname=chckpnt_dirname,
                                  max_epochs=N_EPOCHS,
                                  batch_size=BATCH_SIZE,
                                  is_retrain=IS_RETRAIN,
                                    is_monitor_acc=True,
                                  callbacks=[],
                                  iterator_train__collate_fn=cntxt_trgt_collate(get_cntxt_trgt, is_repeat_batch=True),  
                                  iterator_valid__collate_fn=cntxt_trgt_collate(get_cntxt_trgt_feat),
                                              ))



--- Loading 100%har/transformer_gnp_large_shared_bottleneck ---

100%har/transformer_gnp_large_shared_bottleneck best epoch: 86 val_loss: -8.16725208180622

--- Loading har100%_lab1%_run0/ssl_classifier_gnp_large_shared_bottleneck_finetune_sslonly ---

har100%_lab1%_run0/ssl_classifier_gnp_large_shared_bottleneck_finetune_sslonly best epoch: 1 val_loss: 0.6721114105882182

--- Loading 100%har/transformer_gnp_large_shared_bottleneck ---

100%har/transformer_gnp_large_shared_bottleneck best epoch: 86 val_loss: -8.16725208180622

--- Loading har100%_lab1%_run1/ssl_classifier_gnp_large_shared_bottleneck_finetune_sslonly ---

har100%_lab1%_run1/ssl_classifier_gnp_large_shared_bottleneck_finetune_sslonly best epoch: 1 val_loss: 0.6721089704181528

--- Loading 100%har/transformer_gnp_large_shared_bottleneck ---

100%har/transformer_gnp_large_shared_bottleneck best epoch: 86 val_loss: -8.16725208180622

--- Loading har100%_lab1%_run2/ssl_classifier_gnp_large_shared_bottleneck_finetune_sslonly

In [25]:
import pandas as pd

out = pd.Series({k:v.history[-1]["valid_acc"] for k,v in data_trainers.items()}).reset_index(name="accuracy")
splitted = out["index"].str.split("/", expand = True)
out["meta"] = splitted[0]
out["models"] = splitted[1]

splitted2 = out["meta"].str.split("_run", expand = True)
out["meta"] = splitted2[0]
out["run"] = splitted2[1]

splitted3 = out["meta"].str.split("_lab", expand = True)
out["data sample"] = splitted3[0].str.split("har", expand = True)[1]
out["lab"] = splitted3[1]


out.drop(columns =["index"], inplace = True) 

out.groupby(["models", "lab", "data sample"]).describe()

accuracy  \
                                                                      count   
models                                             lab data sample            
ssl_classifier_gnp_large_shared_bottleneck_fine... 1%  100%             3.0   

                                                                              \
                                                                        mean   
models                                             lab data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 1%  100%         0.876598   

                                                                              \
                                                                         std   
models                                             lab data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 1%  100%         0.004706   

                                                                              \
                                                                         min   
models                                             lab data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 1%  100%         0.871395   

                                                                              \
                                                                         25%   
models                                             lab data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 1%  100%         0.874618   

                                                                              \
                                                                         50%   
models                                             lab data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 1%  100%         0.877842   

                                                                              \
                                                                         75%   
models                                             lab data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 1%  100%         0.879199   

                                                                              
                                                                         max  
models                                             lab data sample            
ssl_classifier_gnp_large_shared_bottleneck_fine... 1%  100%         0.880556

# Sup Only 

In [13]:
from skorch.callbacks import Freezer, LRScheduler

data_trainers = {}

for run in range(3):
    for sampling_perc in [1]:
        for label_perc in [0.01]:
            load_pretrained_(models, [sampling_perc])

            get_lambda_clf=HyperparameterInterpolator(1, 50, N_EPOCHS*n_steps_per_epoch, mode="linear")

            data_train, _, data_test = get_train_dev_test_ssl("har", 
                                                              n_labels=label_perc, 
                                                              data_perc=sampling_perc,
                                                              seed=random.randint(0,10000),
                                                              dev_size=0,
                                                              is_augment=True)

            # add test as unlabeled data
            data_train.data = data_train.data[(data_train.targets!=-1).squeeze()]
            data_train.targets = data_train.targets[(data_train.targets!=-1).squeeze()]

            data_trainers.update(train_models_({"har{}%_lab{}%_run{}".format(int(sampling_perc*100), int(label_perc*100), run):
                                                (data_train, data_test)}, 
                                  {k + "_finetune_suponly" :m for k,m in models.items() if "ssl_classifier" in k}, 
                                  criterion=partial(NeuralProcessSSLLoss, 
                                                    get_lambda_sup=lambda: get_lambda_clf(True),
                                                    n_max_elements=int(128*sampling_perc),
                                                    label_perc=(label_perc * data_train.n_train)/data_train.n_total, # label perc is lower ebcause cocnat to test
                                                    min_sigma=min_std,
                                                    is_unsup_forall=False,
                                                    is_ssl_only=True,
                                                    get_lambda_unsup=lambda: 0,
                                                     get_lambda_ent=lambda: 0.5,  # both do something similar
                                                     get_lambda_neg_cons=lambda: .5,
                                                    ),
                                    patience=15,
                                  chckpnt_dirname=chckpnt_dirname,
                                  max_epochs=N_EPOCHS,
                                  batch_size=BATCH_SIZE,
                                  is_retrain=IS_RETRAIN,
                                               seed=None,
                                    is_monitor_acc=True,
                                  callbacks=[],
                                  iterator_train__collate_fn=cntxt_trgt_collate(get_cntxt_trgt, is_repeat_batch=True),  
                                  iterator_valid__collate_fn=cntxt_trgt_collate(get_cntxt_trgt_feat),
                                              ))



--- Loading 100%har/transformer_gnp_large_shared_bottleneck ---

100%har/transformer_gnp_large_shared_bottleneck best epoch: 86 val_loss: -8.16725208180622

--- Loading har100%_lab1%_run0/ssl_classifier_gnp_large_shared_bottleneck_finetune_suponly ---

har100%_lab1%_run0/ssl_classifier_gnp_large_shared_bottleneck_finetune_suponly best epoch: 2 val_loss: 0.37116682323149275

--- Loading 100%har/transformer_gnp_large_shared_bottleneck ---

100%har/transformer_gnp_large_shared_bottleneck best epoch: 86 val_loss: -8.16725208180622

--- Loading har100%_lab1%_run1/ssl_classifier_gnp_large_shared_bottleneck_finetune_suponly ---

har100%_lab1%_run1/ssl_classifier_gnp_large_shared_bottleneck_finetune_suponly best epoch: 2 val_loss: 0.6973583848750674

--- Loading 100%har/transformer_gnp_large_shared_bottleneck ---

100%har/transformer_gnp_large_shared_bottleneck best epoch: 86 val_loss: -8.16725208180622

--- Loading har100%_lab1%_run2/ssl_classifier_gnp_large_shared_bottleneck_finetune_suponl

In [14]:
import pandas as pd

out = pd.Series({k:v.history[-1]["valid_acc"] for k,v in data_trainers.items()}).reset_index(name="accuracy")
splitted = out["index"].str.split("/", expand = True)
out["meta"] = splitted[0]
out["models"] = splitted[1]

splitted2 = out["meta"].str.split("_run", expand = True)
out["meta"] = splitted2[0]
out["run"] = splitted2[1]

splitted3 = out["meta"].str.split("_lab", expand = True)
out["data sample"] = splitted3[0].str.split("har", expand = True)[1]
out["lab"] = splitted3[1]


out.drop(columns =["index"], inplace = True) 

out.groupby(["models", "lab", "data sample"]).describe()

accuracy  \
                                                                      count   
models                                             lab data sample            
ssl_classifier_gnp_large_shared_bottleneck_fine... 1%  100%             3.0   

                                                                              \
                                                                        mean   
models                                             lab data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 1%  100%         0.904083   

                                                                              \
                                                                         std   
models                                             lab data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 1%  100%         0.013515   

                                                                              \
                                                                         min   
models                                             lab data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 1%  100%         0.892433   

                                                                              \
                                                                         25%   
models                                             lab data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 1%  100%         0.896675   

                                                                              \
                                                                         50%   
models                                             lab data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 1%  100%         0.900916   

                                                                              \
                                                                         75%   
models                                             lab data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 1%  100%         0.909908   

                                                                              
                                                                         max  
models                                             lab data sample            
ssl_classifier_gnp_large_shared_bottleneck_fine... 1%  100%         0.918901

# Sup Only No Scaling

In [16]:
from skorch.callbacks import Freezer, LRScheduler

data_trainers = {}

for run in range(3):
    for sampling_perc in [1]:
        for label_perc in [0.01]:
            load_pretrained_(models, [sampling_perc])

            get_lambda_clf=HyperparameterInterpolator(1, 50, N_EPOCHS*n_steps_per_epoch, mode="linear")

            data_train, _, data_test = get_train_dev_test_ssl("har", 
                                                              n_labels=label_perc, 
                                                              data_perc=sampling_perc,
                                                              seed=random.randint(0,10000),
                                                              dev_size=0,
                                                              is_augment=True)

            # add test as unlabeled data
            data_train.data = data_train.data[(data_train.targets!=-1).squeeze()]
            data_train.targets = data_train.targets[(data_train.targets!=-1).squeeze()]

            data_trainers.update(train_models_({"har{}%_lab{}%_run{}".format(int(sampling_perc*100), int(label_perc*100), run):
                                                (data_train, data_test)}, 
                                  {k + "_finetune_sup_vanilla" :m for k,m in models.items() if "ssl_classifier" in k}, 
                                  criterion=partial(NeuralProcessSSLLoss, 
                                                    get_lambda_sup=lambda : 1,
                                                    n_max_elements=None,
                                                    label_perc=None, # label perc is lower ebcause cocnat to test
                                                    min_sigma=min_std,
                                                    is_unsup_forall=False,
                                                    is_ssl_only=True,
                                                    get_lambda_unsup=lambda: 0,
                                                    ),
                                    patience=15,
                                  chckpnt_dirname=chckpnt_dirname,
                                  max_epochs=N_EPOCHS,
                                  batch_size=BATCH_SIZE,
                                  is_retrain=True,
                                               seed=None,
                                    is_monitor_acc=True,
                                  callbacks=[],
                                  iterator_train__collate_fn=cntxt_trgt_collate(get_cntxt_trgt, is_repeat_batch=True),  
                                  iterator_valid__collate_fn=cntxt_trgt_collate(get_cntxt_trgt_feat),
                                              ))



--- Loading 100%har/transformer_gnp_large_shared_bottleneck ---

100%har/transformer_gnp_large_shared_bottleneck best epoch: 86 val_loss: -8.16725208180622

--- Training har100%_lab1%_run0/ssl_classifier_gnp_large_shared_bottleneck_finetune_sup_vanilla ---



  epoch    train_loss    valid_acc    valid_loss    cp      dur
-------  ------------  -----------  ------------  ----  -------
      1        0.1423       0.9019        0.4632     +  27.0568


      2        0.0234       0.8789        0.6497        25.7552


      3        0.0150       0.9050        0.5428     +  26.0485


      4        0.0204       0.8918        0.5845        26.6478


...

     12        0.0151       0.8758        0.6700        26.0417


     13        0.0265       0.8639        0.8402        26.9045


     14        0.0056       0.8717        0.8625        26.2144


     15        0.0204       0.8775        0.5983        25.7190


Stopping since valid_acc has not improved in the last 15 epochs.
Re-initializing optimizer.
har100%_lab1%_run2/ssl_classifier_gnp_large_shared_bottleneck_finetune_sup_vanilla best epoch: 1 val_loss: 0.4885016896986027


In [17]:
import pandas as pd

out = pd.Series({k:v.history[-1]["valid_acc"] for k,v in data_trainers.items()}).reset_index(name="accuracy")
splitted = out["index"].str.split("/", expand = True)
out["meta"] = splitted[0]
out["models"] = splitted[1]

splitted2 = out["meta"].str.split("_run", expand = True)
out["meta"] = splitted2[0]
out["run"] = splitted2[1]

splitted3 = out["meta"].str.split("_lab", expand = True)
out["data sample"] = splitted3[0].str.split("har", expand = True)[1]
out["lab"] = splitted3[1]


out.drop(columns =["index"], inplace = True) 

out.groupby(["models", "lab", "data sample"]).describe()

accuracy  \
                                                                      count   
models                                             lab data sample            
ssl_classifier_gnp_large_shared_bottleneck_fine... 1%  100%             3.0   

                                                                              \
                                                                        mean   
models                                             lab data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 1%  100%         0.908947   

                                                                              \
                                                                         std   
models                                             lab data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 1%  100%         0.014823   

                                                                              \
                                                                         min   
models                                             lab data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 1%  100%         0.896505   

                                                                              \
                                                                         25%   
models                                             lab data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 1%  100%         0.900747   

                                                                              \
                                                                         50%   
models                                             lab data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 1%  100%         0.904988   

                                                                              \
                                                                         75%   
models                                             lab data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 1%  100%         0.915168   

                                                                              
                                                                         max  
models                                             lab data sample            
ssl_classifier_gnp_large_shared_bottleneck_fine... 1%  100%         0.925348

# No Lambda CLF

In [28]:
from skorch.callbacks import Freezer, LRScheduler

data_trainers = {}

for run in range(3):
    for sampling_perc in [1]:
        for label_perc in [0.01]:
            load_pretrained_(models, [sampling_perc])

            get_lambda_clf=HyperparameterInterpolator(1, 50, N_EPOCHS*n_steps_per_epoch, mode="linear")

            data_train, _, data_test = get_train_dev_test_ssl("har", 
                                                              n_labels=label_perc, 
                                                              data_perc=sampling_perc, 
                                                              dev_size=0,
                                                              seed=random.randint(0,10000),
                                                              is_augment=True)

            # add test as unlabeled data
            data_train.data = np.concatenate([data_train.data, data_test.data], axis=0)
            data_train.targets = np.concatenate([data_train.targets, -1*np.ones_like(data_test.targets)], axis=0)
            data_train.indcs = np.concatenate([data_train.indcs, data_test.indcs], axis=0)

            data_trainers.update(train_models_({"har{}%_lab{}%_run{}".format(int(sampling_perc*100), int(label_perc*100), run): 
                                                (data_train, data_test)}, 
                                  {k + "_finetune_nolambda" :m for k,m in models.items() if "ssl_classifier" in k}, 
                                  criterion=partial(NeuralProcessSSLLoss, 
                                                    get_lambda_sup=lambda: 1,
                                                    n_max_elements=int(128*sampling_perc),
                                                    label_perc=(label_perc * data_train.n_train)/data_train.n_total, # label perc is lower ebcause cocnat to test
                                                    min_sigma=min_std,
                                                    is_unsup_forall=False,
                                                    is_ssl_only=False,
                                                    get_lambda_unsup=lambda: 1,
                                                     get_lambda_ent=lambda: 0.5,  # both do something similar
                                                     get_lambda_neg_cons=lambda: .5,
                                                    ),
                                    patience=15,
                                  chckpnt_dirname=chckpnt_dirname,
                                  max_epochs=N_EPOCHS,
                                  batch_size=BATCH_SIZE,
                                  is_retrain=IS_RETRAIN,
                                    is_monitor_acc=True,
                                  callbacks=[],
                                  iterator_train__collate_fn=cntxt_trgt_collate(get_cntxt_trgt, is_repeat_batch=True),  
                                  iterator_valid__collate_fn=cntxt_trgt_collate(get_cntxt_trgt_feat),
                                              ))



--- Loading 100%har/transformer_gnp_large_shared_bottleneck ---

100%har/transformer_gnp_large_shared_bottleneck best epoch: 86 val_loss: -8.16725208180622

--- Loading har100%_lab1%_run0/ssl_classifier_gnp_large_shared_bottleneck_finetune_nolambda ---

har100%_lab1%_run0/ssl_classifier_gnp_large_shared_bottleneck_finetune_nolambda best epoch: 4 val_loss: 0.7258332047496442

--- Loading 100%har/transformer_gnp_large_shared_bottleneck ---

100%har/transformer_gnp_large_shared_bottleneck best epoch: 86 val_loss: -8.16725208180622

--- Loading har100%_lab1%_run1/ssl_classifier_gnp_large_shared_bottleneck_finetune_nolambda ---

har100%_lab1%_run1/ssl_classifier_gnp_large_shared_bottleneck_finetune_nolambda best epoch: 2 val_loss: 0.8610166228339354

--- Loading 100%har/transformer_gnp_large_shared_bottleneck ---

100%har/transformer_gnp_large_shared_bottleneck best epoch: 86 val_loss: -8.16725208180622

--- Loading har100%_lab1%_run2/ssl_classifier_gnp_large_shared_bottleneck_finetune_nol

In [29]:
import pandas as pd

out = pd.Series({k:v.history[-1]["valid_acc"] for k,v in data_trainers.items()}).reset_index(name="accuracy")
splitted = out["index"].str.split("/", expand = True)
out["meta"] = splitted[0]
out["models"] = splitted[1]

splitted2 = out["meta"].str.split("_run", expand = True)
out["meta"] = splitted2[0]
out["run"] = splitted2[1]

splitted3 = out["meta"].str.split("_lab", expand = True)
out["data sample"] = splitted3[0].str.split("har", expand = True)[1]
out["lab"] = splitted3[1]


out.drop(columns =["index"], inplace = True) 

out.groupby(["models", "lab", "data sample"]).describe()

accuracy  \
                                                                      count   
models                                             lab data sample            
ssl_classifier_gnp_large_shared_bottleneck_fine... 1%  100%             3.0   

                                                                             \
                                                                       mean   
models                                             lab data sample            
ssl_classifier_gnp_large_shared_bottleneck_fine... 1%  100%         0.88542   

                                                                            \
                                                                       std   
models                                             lab data sample           
ssl_classifier_gnp_large_shared_bottleneck_fine... 1%  100%         0.0118   

                                                                              \
                                                                         min   
models                                             lab data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 1%  100%         0.872073   

                                                                              \
                                                                         25%   
models                                             lab data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 1%  100%         0.880896   

                                                                              \
                                                                         50%   
models                                             lab data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 1%  100%         0.889718   

                                                                              \
                                                                         75%   
models                                             lab data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 1%  100%         0.892094   

                                                                              
                                                                         max  
models                                             lab data sample            
ssl_classifier_gnp_large_shared_bottleneck_fine... 1%  100%         0.894469

## Without Label Scaling

In [30]:
from skorch.callbacks import Freezer, LRScheduler

data_trainers = {}

for run in range(3):
    for sampling_perc in [1]:
        for label_perc in [0.01]:
            load_pretrained_(models, [sampling_perc])

            get_lambda_clf=HyperparameterInterpolator(1, 50, N_EPOCHS*n_steps_per_epoch, mode="linear")

            data_train, _, data_test = get_train_dev_test_ssl("har", 
                                                              n_labels=label_perc, 
                                                              data_perc=sampling_perc, 
                                                              dev_size=0,
                                                              seed=random.randint(0,10000),
                                                              is_augment=True)

            # add test as unlabeled data
            data_train.data = np.concatenate([data_train.data, data_test.data], axis=0)
            data_train.targets = np.concatenate([data_train.targets, -1*np.ones_like(data_test.targets)], axis=0)
            data_train.indcs = np.concatenate([data_train.indcs, data_test.indcs], axis=0)

            data_trainers.update(train_models_({"har{}%_lab{}%_run{}".format(int(sampling_perc*100), int(label_perc*100), run): 
                                                (data_train, data_test)}, 
                                  {k + "_finetune_nolabscale" :m for k,m in models.items() if "ssl_classifier" in k}, 
                                  criterion=partial(NeuralProcessSSLLoss, 
                                                    get_lambda_sup=lambda: get_lambda_clf(True),
                                                    n_max_elements=int(128*sampling_perc),
                                                    label_perc=None, # label perc is lower ebcause cocnat to test
                                                    min_sigma=min_std,
                                                    is_unsup_forall=False,
                                                    is_ssl_only=False,
                                                    get_lambda_unsup=lambda: 1,
                                                     get_lambda_ent=lambda: 0.5,  # both do something similar
                                                     get_lambda_neg_cons=lambda: 0.5,
                                                    ),
                                    patience=15,
                                  chckpnt_dirname=chckpnt_dirname,
                                  max_epochs=N_EPOCHS,
                                  batch_size=BATCH_SIZE,
                                  is_retrain=IS_RETRAIN,
                                    is_monitor_acc=True,
                                  callbacks=[],
                                  iterator_train__collate_fn=cntxt_trgt_collate(get_cntxt_trgt, is_repeat_batch=True),  
                                  iterator_valid__collate_fn=cntxt_trgt_collate(get_cntxt_trgt_feat),
                                              ))



--- Loading 100%har/transformer_gnp_large_shared_bottleneck ---

100%har/transformer_gnp_large_shared_bottleneck best epoch: 86 val_loss: -8.16725208180622

--- Loading har100%_lab1%_run0/ssl_classifier_gnp_large_shared_bottleneck_finetune_nolabscale ---

har100%_lab1%_run0/ssl_classifier_gnp_large_shared_bottleneck_finetune_nolabscale best epoch: 1 val_loss: 1.3277767340450073

--- Loading 100%har/transformer_gnp_large_shared_bottleneck ---

100%har/transformer_gnp_large_shared_bottleneck best epoch: 86 val_loss: -8.16725208180622

--- Loading har100%_lab1%_run1/ssl_classifier_gnp_large_shared_bottleneck_finetune_nolabscale ---

har100%_lab1%_run1/ssl_classifier_gnp_large_shared_bottleneck_finetune_nolabscale best epoch: 2 val_loss: 1.1983000453059698

--- Loading 100%har/transformer_gnp_large_shared_bottleneck ---

100%har/transformer_gnp_large_shared_bottleneck best epoch: 86 val_loss: -8.16725208180622

--- Loading har100%_lab1%_run2/ssl_classifier_gnp_large_shared_bottleneck_fine

In [31]:
import pandas as pd

out = pd.Series({k:v.history[-1]["valid_acc"] for k,v in data_trainers.items()}).reset_index(name="accuracy")
splitted = out["index"].str.split("/", expand = True)
out["meta"] = splitted[0]
out["models"] = splitted[1]

splitted2 = out["meta"].str.split("_run", expand = True)
out["meta"] = splitted2[0]
out["run"] = splitted2[1]

splitted3 = out["meta"].str.split("_lab", expand = True)
out["data sample"] = splitted3[0].str.split("har", expand = True)[1]
out["lab"] = splitted3[1]


out.drop(columns =["index"], inplace = True) 

out.groupby(["models", "lab", "data sample"]).describe()

accuracy  \
                                                                      count   
models                                             lab data sample            
ssl_classifier_gnp_large_shared_bottleneck_fine... 1%  100%             3.0   

                                                                              \
                                                                        mean   
models                                             lab data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 1%  100%         0.893677   

                                                                              \
                                                                         std   
models                                             lab data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 1%  100%         0.012967   

                                                                              \
                                                                         min   
models                                             lab data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 1%  100%         0.881574   

                                                                              \
                                                                         25%   
models                                             lab data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 1%  100%         0.886834   

                                                                              \
                                                                         50%   
models                                             lab data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 1%  100%         0.892094   

                                                                              \
                                                                         75%   
models                                             lab data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 1%  100%         0.899729   

                                                                              
                                                                         max  
models                                             lab data sample            
ssl_classifier_gnp_large_shared_bottleneck_fine... 1%  100%         0.907363

## No Element Scaling

In [32]:
from skorch.callbacks import Freezer, LRScheduler

data_trainers = {}

for run in range(3):
    for sampling_perc in [1]:
        for label_perc in [0.01]:

            load_pretrained_(models, [sampling_perc])

            get_lambda_clf=HyperparameterInterpolator(1, 50, N_EPOCHS*n_steps_per_epoch, mode="linear")

            data_train, _, data_test = get_train_dev_test_ssl("har", 
                                                              n_labels=label_perc, 
                                                              data_perc=sampling_perc, 
                                                              dev_size=0,
                                                              seed=random.randint(0,10000),
                                                              is_augment=True)

            # add test as unlabeled data
            data_train.data = np.concatenate([data_train.data, data_test.data], axis=0)
            data_train.targets = np.concatenate([data_train.targets, -1*np.ones_like(data_test.targets)], axis=0)
            data_train.indcs = np.concatenate([data_train.indcs, data_test.indcs], axis=0)

            data_trainers.update(train_models_({"har{}%_lab{}%_run{}".format(int(sampling_perc*100), int(label_perc*100), run): 
                                                (data_train, data_test)}, 
                                  {k + "_finetune_noelemscale" :m for k,m in models.items() if "ssl_classifier" in k}, 
                                  criterion=partial(NeuralProcessSSLLoss, 
                                                    get_lambda_sup=lambda: get_lambda_clf(True),
                                                    n_max_elements=None,
                                                    label_perc=(label_perc * data_train.n_train)/data_train.n_total, # label perc is lower ebcause cocnat to test
                                                    min_sigma=min_std,
                                                    is_unsup_forall=False,
                                                    is_ssl_only=False,
                                                    get_lambda_unsup=lambda: 1,
                                                     get_lambda_ent=lambda: 0.5,  # both do something similar
                                                     get_lambda_neg_cons=lambda: 0.5,
                                                    ),
                                    patience=15,
                                  chckpnt_dirname=chckpnt_dirname,
                                  max_epochs=N_EPOCHS,
                                  batch_size=BATCH_SIZE,
                                  is_retrain=IS_RETRAIN,
                                    is_monitor_acc=True,
                                  callbacks=[],
                                  iterator_train__collate_fn=cntxt_trgt_collate(get_cntxt_trgt, is_repeat_batch=True),  
                                  iterator_valid__collate_fn=cntxt_trgt_collate(get_cntxt_trgt_feat),
                                              ))



--- Loading 100%har/transformer_gnp_large_shared_bottleneck ---

100%har/transformer_gnp_large_shared_bottleneck best epoch: 86 val_loss: -8.16725208180622

--- Loading har100%_lab1%_run0/ssl_classifier_gnp_large_shared_bottleneck_finetune_noelemscale ---

har100%_lab1%_run0/ssl_classifier_gnp_large_shared_bottleneck_finetune_noelemscale best epoch: 1 val_loss: 0.5697207075480812

--- Loading 100%har/transformer_gnp_large_shared_bottleneck ---

100%har/transformer_gnp_large_shared_bottleneck best epoch: 86 val_loss: -8.16725208180622

--- Loading har100%_lab1%_run1/ssl_classifier_gnp_large_shared_bottleneck_finetune_noelemscale ---

har100%_lab1%_run1/ssl_classifier_gnp_large_shared_bottleneck_finetune_noelemscale best epoch: 2 val_loss: 0.7624972491981142

--- Loading 100%har/transformer_gnp_large_shared_bottleneck ---

100%har/transformer_gnp_large_shared_bottleneck best epoch: 86 val_loss: -8.16725208180622

--- Loading har100%_lab1%_run2/ssl_classifier_gnp_large_shared_bottleneck_

In [33]:
import pandas as pd

out = pd.Series({k:v.history[-1]["valid_acc"] for k,v in data_trainers.items()}).reset_index(name="accuracy")
splitted = out["index"].str.split("/", expand = True)
out["meta"] = splitted[0]
out["models"] = splitted[1]

splitted2 = out["meta"].str.split("_run", expand = True)
out["meta"] = splitted2[0]
out["run"] = splitted2[1]

splitted3 = out["meta"].str.split("_lab", expand = True)
out["data sample"] = splitted3[0].str.split("har", expand = True)[1]
out["lab"] = splitted3[1]


out.drop(columns =["index"], inplace = True) 

out.groupby(["models", "lab", "data sample"]).describe()

accuracy  \
                                                                      count   
models                                             lab data sample            
ssl_classifier_gnp_large_shared_bottleneck_fine... 1%  100%             3.0   

                                                                              \
                                                                        mean   
models                                             lab data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 1%  100%         0.882366   

                                                                              \
                                                                         std   
models                                             lab data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 1%  100%         0.003628   

                                                                              \
                                                                         min   
models                                             lab data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 1%  100%         0.878181   

                                                                              \
                                                                         25%   
models                                             lab data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 1%  100%         0.881235   

                                                                              \
                                                                         50%   
models                                             lab data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 1%  100%         0.884289   

                                                                              \
                                                                         75%   
models                                             lab data sample             
ssl_classifier_gnp_large_shared_bottleneck_fine... 1%  100%         0.884459   

                                                                              
                                                                         max  
models                                             lab data sample            
ssl_classifier_gnp_large_shared_bottleneck_fine... 1%  100%         0.884628

## No Finetuning

In [34]:
from skorch.callbacks import Freezer, LRScheduler

data_trainers = {}
models["ssl_classifier_gnp_large_shared_bottleneck"] = partial(GlobalNeuralProcess, **kwargs)

for run in range(3):
    for sampling_perc in [1]:
        for label_perc in [0.01]:

            get_lambda_clf=HyperparameterInterpolator(1, 50, N_EPOCHS*n_steps_per_epoch, mode="linear")

            data_train, _, data_test = get_train_dev_test_ssl("har", 
                                                              n_labels=label_perc, 
                                                              data_perc=sampling_perc, 
                                                              dev_size=0,
                                                              seed=random.randint(0,10000),
                                                              is_augment=True)

            # add test as unlabeled data
            data_train.data = np.concatenate([data_train.data, data_test.data], axis=0)
            data_train.targets = np.concatenate([data_train.targets, -1*np.ones_like(data_test.targets)], axis=0)
            data_train.indcs = np.concatenate([data_train.indcs, data_test.indcs], axis=0)

            data_trainers.update(train_models_({"har{}%_lab{}%_run{}".format(int(sampling_perc*100), int(label_perc*100), run): 
                                                (data_train, data_test)}, 
                                  {k + "" :m for k,m in models.items() if "ssl_classifier" in k}, 
                                  criterion=partial(NeuralProcessSSLLoss, 
                                                    get_lambda_sup=lambda: get_lambda_clf(True),
                                                    n_max_elements=int(128*sampling_perc),
                                                    label_perc=(label_perc * data_train.n_train)/data_train.n_total, # label perc is lower ebcause cocnat to test
                                                    min_sigma=min_std,
                                                    is_unsup_forall=False,
                                                    is_ssl_only=False,
                                                    get_lambda_unsup=lambda: 1,
                                                     get_lambda_ent=lambda: 0.5,  # both do something similar
                                                     get_lambda_neg_cons=lambda: 0.5,
                                                    ),
                                    patience=15,
                                  chckpnt_dirname=chckpnt_dirname,
                                  max_epochs=N_EPOCHS,
                                  batch_size=BATCH_SIZE,
                                  is_retrain=IS_RETRAIN,
                                    is_monitor_acc=True,
                                  callbacks=[],
                                  iterator_train__collate_fn=cntxt_trgt_collate(get_cntxt_trgt, is_repeat_batch=True),  
                                  iterator_valid__collate_fn=cntxt_trgt_collate(get_cntxt_trgt_feat),
                                              ))



--- Loading har100%_lab1%_run0/ssl_classifier_gnp_large_shared_bottleneck ---

har100%_lab1%_run0/ssl_classifier_gnp_large_shared_bottleneck best epoch: 1 val_loss: 0.8911615639651959

--- Loading har100%_lab1%_run1/ssl_classifier_gnp_large_shared_bottleneck ---

har100%_lab1%_run1/ssl_classifier_gnp_large_shared_bottleneck best epoch: 2 val_loss: 0.5388232102667948

--- Loading har100%_lab1%_run2/ssl_classifier_gnp_large_shared_bottleneck ---

har100%_lab1%_run2/ssl_classifier_gnp_large_shared_bottleneck best epoch: 1 val_loss: 0.8988694104252315


In [35]:
import pandas as pd

out = pd.Series({k:v.history[-1]["valid_acc"] for k,v in data_trainers.items()}).reset_index(name="accuracy")
splitted = out["index"].str.split("/", expand = True)
out["meta"] = splitted[0]
out["models"] = splitted[1]

splitted2 = out["meta"].str.split("_run", expand = True)
out["meta"] = splitted2[0]
out["run"] = splitted2[1]

splitted3 = out["meta"].str.split("_lab", expand = True)
out["data sample"] = splitted3[0].str.split("har", expand = True)[1]
out["lab"] = splitted3[1]


out.drop(columns =["index"], inplace = True) 

out.groupby(["models", "lab", "data sample"]).describe()

accuracy            \
                                                              count      mean   
models                                     lab data sample                      
ssl_classifier_gnp_large_shared_bottleneck 1%  100%             3.0  0.862007   

                                                                      \
                                                                 std   
models                                     lab data sample             
ssl_classifier_gnp_large_shared_bottleneck 1%  100%         0.017089   

                                                                      \
                                                                 min   
models                                     lab data sample             
ssl_classifier_gnp_large_shared_bottleneck 1%  100%         0.850017   

                                                                      \
                                                                 25%   
models                                     lab data sample             
ssl_classifier_gnp_large_shared_bottleneck 1%  100%         0.852223   

                                                                      \
                                                                 50%   
models                                     lab data sample             
ssl_classifier_gnp_large_shared_bottleneck 1%  100%         0.854428   

                                                                                
                                                                 75%       max  
models                                     lab data sample                      
ssl_classifier_gnp_large_shared_bottleneck 1%  100%         0.868001  0.881574